In [ ]:
# get successful connection with TD ameritrade
# required: acct number - working API key
# get live data from TD first

# build sim for simulating trades


In [1]:

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os


def find(name, path):
    for root, dirs, files in os.walk(path):
        if name in files:
            return os.path.join(root, name)

# Load the data from the Excel file
x = find('bb_crude_data.xlsx', os.getcwd())
original_data = pd.read_excel(x)
original_data.columns.tolist()
original_data = original_data.set_index('Dates')
original_data = original_data[['Open', 'High', 'Low', 'Close']]
original_data = original_data.dropna()

original_data.head()


,Open,High,Low,Close
Dates,,,,
2010-04-13,84.36,84.42,82.51,84.05
2010-04-14,83.82,86.39,83.71,85.84
2010-04-15,85.91,86.27,85.27,85.51
2010-04-16,85.31,85.44,82.52,83.24
2010-04-19,82.90,83.00,80.53,81.45


In [2]:
from backtesting import Backtest, Strategy
from backtesting.lib import crossover

from backtesting.test import SMA, GOOG


class SmaCross(Strategy):
    n1 = 9
    n2 = 21

    def init(self):
        close = self.data.Close
        self.sma1 = self.I(SMA, close, self.n1)
        self.sma2 = self.I(SMA, close, self.n2)

    def next(self):
        if crossover(self.sma1, self.sma2):
            self.buy()
        elif crossover(self.sma2, self.sma1):
            self.sell()


bt = Backtest(original_data, SmaCross,
              cash=100000, commission=.002,
              exclusive_orders=True)

output = bt.run()
bt.plot()

C:\Users\rrain\AppData\Local\Programs\Python\Python311\Lib\site-packages\backtesting\_plotting.py:50: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

C:\Users\rrain\AppData\Local\Programs\Python\Python311\Lib\site-packages\backtesting\test\__init__.py:8: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  return pd.read_csv(join(dirname(__file__), filename),
C:\Users\rrain\AppData\Local\Programs\Python\Python311\Lib\site-packages\backtesting\test\__init__.py:8: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  return pd.read_csv(join(dirname(__file__), filename),
C:\Users\rrain\AppData\Local\Programs\Python\Python311\Lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now onl

GridPlot(id='p1433', ...)

In [3]:
import pandas as pd


def SMA(array, n):
    """Simple moving average"""
    return pd.Series(array).rolling(n).mean()


def RSI(array, n):
    """Relative strength index"""
    # Approximate; good enough
    gain = pd.Series(array).diff()
    loss = gain.copy()
    gain[gain < 0] = 0
    loss[loss > 0] = 0
    rs = gain.ewm(n).mean() / loss.abs().ewm(n).mean()
    return 100 - 100 / (1 + rs)


from backtesting import Strategy, Backtest
from backtesting.lib import resample_apply


class System(Strategy):
    d_rsi = 30  # Daily RSI lookback periods
    w_rsi = 30  # Weekly
    level = 70
    
    def init(self):
        # Compute moving averages the strategy demands
        self.ma10 = self.I(SMA, self.data.Close, 10)
        self.ma20 = self.I(SMA, self.data.Close, 20)
        self.ma50 = self.I(SMA, self.data.Close, 50)
        self.ma100 = self.I(SMA, self.data.Close, 100)
        
        # Compute daily RSI(30)
        self.daily_rsi = self.I(RSI, self.data.Close, self.d_rsi)
        
        # To construct weekly RSI, we can use `resample_apply()`
        # helper function from the library
        self.weekly_rsi = resample_apply(
            'W-FRI', RSI, self.data.Close, self.w_rsi)
        
        
    def next(self):
        price = self.data.Close[-1]
        
        # If we don't already have a position, and
        # if all conditions are satisfied, enter long.
        if (not self.position and
            self.daily_rsi[-1] > self.level and
            self.weekly_rsi[-1] > self.level and
            self.weekly_rsi[-1] > self.daily_rsi[-1] and
            self.ma10[-1] > self.ma20[-1] > self.ma50[-1] > self.ma100[-1] and
            price > self.ma10[-1]):
            
            # Buy at market price on next open, but do
            # set 8% fixed stop loss.
            self.buy(sl=.92 * price)
        
        # If the price closes 2% or more below 10-day MA
        # close the position, if any.
        elif price < .98 * self.ma10[-1]:
            self.position.close()

In [4]:
from backtesting.test import GOOG

backtest = Backtest(GOOG, System, commission=.002)
backtest.run()

Start                     2004-08-19 00:00:00
End                       2013-03-01 00:00:00
Duration                   3116 days 00:00:00
Exposure Time [%]                    2.793296
Equity Final [$]                  10017.44422
Equity Peak [$]                    10978.3801
Return [%]                           0.174442
Buy & Hold Return [%]              703.458242
Return (Ann.) [%]                     0.02045
Volatility (Ann.) [%]                4.941212
Sharpe Ratio                         0.004139
Sortino Ratio                         0.00536
Calmar Ratio                         0.002043
Max. Drawdown [%]                   -10.00745
Avg. Drawdown [%]                   -9.340092
Max. Drawdown Duration     2653 days 00:00:00
Avg. Drawdown Duration     1410 days 00:00:00
# Trades                                    4
Win Rate [%]                             25.0
Best Trade [%]                       9.687579
Worst Trade [%]                     -4.456159
Avg. Trade [%]                    

In [5]:
%%time

backtest.optimize(d_rsi=range(10, 35, 5),
                  w_rsi=range(10, 35, 5),
                  level=range(30, 80, 10))


CPU times: total: 7.64 s
Wall time: 7.66 s


Start                     2004-08-19 00:00:00
End                       2013-03-01 00:00:00
Duration                   3116 days 00:00:00
Exposure Time [%]                   22.486034
Equity Final [$]                  22587.83224
Equity Peak [$]                   23395.59144
Return [%]                         125.878322
Buy & Hold Return [%]              703.458242
Return (Ann.) [%]                    10.03124
Volatility (Ann.) [%]               13.124247
Sharpe Ratio                         0.764329
Sortino Ratio                         1.28711
Calmar Ratio                         0.530172
Max. Drawdown [%]                  -18.920719
Avg. Drawdown [%]                   -3.795058
Max. Drawdown Duration      778 days 00:00:00
Avg. Drawdown Duration       97 days 00:00:00
# Trades                                   23
Win Rate [%]                        65.217391
Best Trade [%]                      25.034669
Worst Trade [%]                     -6.297769
Avg. Trade [%]                    